In [300]:
%load_ext autoreload
%autoreload 2
    
import sys

sys.path.append("../lib/")

# from etna.experimental.classification.feature_extraction import TSFreshFeatureExtractor
from tsfresh import extract_features
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostClassifier, Pool
import numpy as np
from sklearn.metrics import classification_report

tqdm.pandas()

import prepare_dataset

/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:362: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:362: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with th

In [169]:
weather_train_path = "../data/weather_train.csv"

In [170]:
weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)

<ipython-input-3-3a06d231aa5a>:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_train = pd.read_csv(weather_train_path, sep=";", index_col=0)


In [ ]:
weather_train_x, weather_train_y = prepare_dataset.prepare_dataset(weather_train)

In [184]:
one_station_x = weather_train_x[weather_train.station_id == 34123]
one_station_y = weather_train_y[weather_train.station_id == 34123]

In [185]:
one_station_x.sort_values("local_time", inplace=True)
one_station_y = one_station_y.loc[one_station_x.index]
one_station_x.reset_index(drop=True, inplace=True)
one_station_y.reset_index(drop=True, inplace=True)

<ipython-input-18-4b7d8955b09c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_station_x.sort_values("local_time", inplace=True)


In [180]:
one_station.columns

Index(['local_time', 'T', 'Po', 'P', 'Pa', 'U', 'DD', 'Ff', 'ff10', 'ff3', 'N',
       'WW', 'W1', 'W2', 'Tn', 'Tx', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV', 'Td',
       'RRR', 'tR', 'E', 'Tg', 'E'', 'sss', 'station_id'],
      dtype='object')

In [305]:
cat_features = ['DD', 'N', 'Cl', 'Nh', 'H', 'Cm', 'Ch', 'VV', 'RRR', 'E', "E'"]
num_features = set(one_station.columns) - set(cat_features) - {"local_time", "station_id", "WW", "W1", "W2", "sss"}

In [191]:
labels = one_station_y.columns

In [202]:
united = pd.concat([one_station_x, one_station_y], axis=1)
for label in 
united.rename({label: "feat_" + label for label in labels}, axis=1, inplace=True)


In [277]:
y = one_station_y.iloc[1:].reset_index(drop=True)
x = united.iloc[:-1].reset_index(drop=True)

In [303]:
X_train = x.iloc[:45000]
y_train = y.iloc[:45000]

X_test = x.iloc[45000:]
y_test = y.iloc[45000:]

In [279]:
X_train["sss"].unique()

array([nan, '35.0', '34.0', '33.0', '32.0', '31.0', '30.0', '29.0',
       '28.0', '27.0', '25.0', '26.0', '36.0', '37.0', '39.0', '24.0',
       '23.0', '19.0', '16.0', '12.0', '9.0', '6.0', '0.0', '4.0', '5.0',
       '7.0', '8.0', '1.0', '3.0', '2.0', '21.0', '46.0', '47.0', '49.0',
       '52.0', '53.0', '54.0', '58.0', '59.0', '60.0', '55.0', '62.0',
       '64.0', '66.0', '65.0', '67.0', '70.0', '69.0', '68.0', '63.0',
       '61.0', '57.0', '56.0', '48.0', '40.0', '15.0', '18.0', '14.0',
       '10.0', '11.0', '17.0', '13.0', '20.0', '22.0', '45.0', '43.0',
       '38.0', '44.0', '42.0', 'Снежный покров не постоянный.',
       'Менее 0.5', '41.0', '51.0', '50.0'], dtype=object)

In [313]:
y_train = y_train.drop(columns=['SHKVAL'])

In [314]:
y_train.astype(int)

,VETER,METEL,DOZD,SNEG,GRAD,TUMAN,GOLOLED
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,1,1
3,0,0,0,0,0,1,1
4,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...
44995,0,0,0,0,0,0,0
44996,0,0,0,0,0,0,0
44997,0,0,0,0,0,0,0
44998,0,0,0,0,0,0,0


In [315]:
train_pool = Pool(X_train[num_features], y_train.astype(int))

<ipython-input-16-2cf6a2976a4a>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_pool = Pool(X_train[num_features], y_train.astype(int))


In [316]:
model = CatBoostClassifier(loss_function='MultiLogloss')
model.fit(train_pool, verbose=0)

In [350]:
y_pred = model.predict(X_test[num_features])

<ipython-input-51-7fa66211aa6e>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  y_pred = model.predict(X_test[num_features])


In [341]:
y_test_check = y_test.drop(columns=['SHKVAL']).astype(int).values

acc = [np.array_equal(y_test_check[i], y_pred[i]) for i in range(len(y_pred))]

In [344]:
np.mean(acc)

0.7206089128299294

In [349]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [353]:
np.mean(y_pred == y_test.drop(columns=['SHKVAL']).astype(int))

/home/jupyter/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3417: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


VETER      0.999965
METEL      0.994363
DOZD       0.824103
SNEG       0.888558
GRAD       0.996607
TUMAN      0.969086
GOLOLED    0.995811
dtype: float64

In [317]:
y_pred = [x == "True" for x in model.predict(X_test[num_features].fillna(0), verbose=0)]

<ipython-input-18-68024bed65c7>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  y_pred = [x == "True" for x in model.predict(X_test[num_features].fillna(0), verbose=0)]
<ipython-input-18-68024bed65c7>:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  y_pred = [x == "True" for x in model.predict(X_test[num_features].fillna(0), verbose=0)]


In [ ]:
np.mean(y_test == y_pred)

In [359]:
for idx, lbl in enumerate(y_test.drop("SHKVAL", axis=1).columns):
    print("Report for", lbl)
    print(classification_report(y_test.loc[:, lbl].astype(int), y_pred[:, idx]))

Report for VETER


/home/jupyter/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jupyter/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    311495
           1       0.00      0.00      0.00        11

    accuracy                           1.00    311506
   macro avg       0.50      0.50      0.50    311506
weighted avg       1.00      1.00      1.00    311506

Report for METEL


/home/jupyter/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00    309757
           1       0.32      0.00      0.01      1749

    accuracy                           0.99    311506
   macro avg       0.66      0.50      0.50    311506
weighted avg       0.99      0.99      0.99    311506

Report for DOZD
              precision    recall  f1-score   support

           0       0.85      0.94      0.89    245274
           1       0.64      0.39      0.49     66232

    accuracy                           0.82    311506
   macro avg       0.75      0.67      0.69    311506
weighted avg       0.81      0.82      0.81    311506

Report for SNEG
              precision    recall  f1-score   support

           0       0.94      0.94      0.94    273924
           1       0.54      0.55      0.54     37582

    accuracy                           0.89    311506
   macro avg       0.74      0.74      0.74    311506
weighted avg       0.89      0.89      0.8